# Course9 Week4 Assignment

## Comparison of London and Paris

### 0. Preparation

In [1]:
#import library
import numpy as np
import pandas as pd
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
! pip install folium==0.5.0
import folium # plotting library
from sklearn.cluster import KMeans
print('installation is done')

installation is done


In [2]:
from arcgis.geocoding import geocode
from arcgis.gis import GIS
gis = GIS()
print('imported')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/arcgis/features/_data/geodataset/utils.py:16: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime,


imported


In [3]:
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline  
from sklearn.datasets.samples_generator import make_blobs

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [5]:
#Define FourSquare Credentials and Version
CLIENT_ID = 'LNK1UWBA5Y01WH44DTFMCGT4DZ4NLWOEP5YOFVYXGLGROD5Z' # your Foursquare ID
CLIENT_SECRET = 'Y0XT5AJR0SBKSBFEPZWR252RFSEV1INHGMOBOVSJ001ZDNOG' # your Foursquare Secret
ACCESS_TOKEN = '23OW2MCXF0NG4HKJCUJL1R1W2QQXHL13B00X2BKKYCQJSIEK' # your FourSquare Access Token
#(copied from server as a note: access_token"23OW2MCXF0NG4HKJCUJL1R1W2QQXHL13B00X2BKKYCQJSIEK")
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LNK1UWBA5Y01WH44DTFMCGT4DZ4NLWOEP5YOFVYXGLGROD5Z
CLIENT_SECRET:Y0XT5AJR0SBKSBFEPZWR252RFSEV1INHGMOBOVSJ001ZDNOG


In [6]:
#link to London data
url_london = 'https://raw.githubusercontent.com/sjwhitworth/london_geojson/01d74d7bba3fbf4c2e3b70ab34c810f7e09964ff/london_postcodes.json'

#link to Paris data
url_paris = 'https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e'

### 1. Download and Preprocessing Dataset

#### 1.1 London

In [7]:
#London data
!wget -q -O 'London_data.json' https://raw.githubusercontent.com/sjwhitworth/london_geojson/01d74d7bba3fbf4c2e3b70ab34c810f7e09964ff/london_postcodes.json
print('Data downloaded')

Data downloaded


In [8]:
with open('London_data.json') as json_data:
    london_data = json.load(json_data)

In [9]:
neighborhoods_data = london_data['features']

In [10]:
columns_names = ['Post_code', 'Town','Latitude', 'Longitude']
london_data3 = pd.DataFrame(columns=columns_names)
london_data3

,Post_code,Town,Latitude,Longitude


In [11]:
for data in neighborhoods_data:
    Name = neighborhood_name = data['properties']['Name'] 
    neighborhood_name = 'London'
        
    neighborhood_latlon = data['geometry']['coordinates'][0][0]
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    london_data3 = london_data3.append({'Post_code': Name,
                                          'Town': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
london_data3

,Post_code,Town,Latitude,Longitude
0,E2,London,51.53179,-0.04385
1,E3,London,51.53952,-0.03401
2,E4,London,51.63223,0.02200
3,E5,London,51.55989,-0.03337
4,E6,London,51.53823,0.04204
...,...,...,...,...
172,WC2B,London,51.51490,-0.11847
173,WC2E,London,51.51091,-0.12033
174,WC2H,London,51.51265,-0.12908
175,WC2N,London,51.51025,-0.12780


#### 1.2 Paris

In [23]:
#scrape data of Paris
!wget -q -O 'france-data.json' https://www.data.gouv.fr/fr/datasets/r/e88c6fda-1d09-42a0-a069-606d3259114e
print('Data downloaded')
paris_raw = pd.read_json('france-data.json')

Data downloaded


In [14]:
paris_data = pd.DataFrame()
for i in paris_raw.fields:
    dict_new = i
    paris_data = paris_data.append(dict_new, ignore_index=True)

In [15]:
#select the necessary columns
paris_data2 = paris_data[['postal_code','nom_comm','nom_dept','geo_point_2d']]
paris_data2

,postal_code,nom_comm,nom_dept,geo_point_2d
0,91370,VERRIERES-LE-BUISSON,ESSONNE,"[48.750443119964764, 2.251712972144151]"
1,77126,COURCELLES-EN-BASSEE,SEINE-ET-MARNE,"[48.41256065214989, 3.052940505560729]"
2,91730,MAUCHAMPS,ESSONNE,"[48.52726809075556, 2.19718165044305]"
3,77400,LAGNY-SUR-MARNE,SEINE-ET-MARNE,"[48.87307018579678, 2.7097808131278462]"
4,94110,ARCUEIL,VAL-DE-MARNE,"[48.80588035965699, 2.333510249842654]"
...,...,...,...,...
1295,77520,CESSOY-EN-MONTOIS,SEINE-ET-MARNE,"[48.50730730461658, 3.138844194183689]"
1296,93420,VILLEPINTE,SEINE-SAINT-DENIS,"[48.95902025378707, 2.536306342059409]"
1297,77130,CANNES-ECLUSE,SEINE-ET-MARNE,"[48.36403767307805, 2.990786679832767]"
1298,78930,VILLETTE,YVELINES,"[48.92627887061508, 1.6937417245662671]"


In [16]:
paris_data2 = paris_data2[paris_data2['nom_dept'].str.contains('PARIS')].reset_index(drop=True)

,postal_code,nom_comm,nom_dept,geo_point_2d
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,"[48.87689616237872, 2.337460241388529]"
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,"[48.86790337886785, 2.344107166658533]"
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,"[48.85941549762748, 2.378741060237548]"
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,"[48.86305413181178, 2.359361058970589]"
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,"[48.84896809191946, 2.332670898588416]"


In [24]:
#prepare a new table for lat and lon seprated one
columns_names = ['Post_code','Name','Town','Latitude','Longitude']
paris_data3 = pd.DataFrame(columns=columns_names)

In [50]:
#extracting Lat Lon -- Start
lis_lat = []
for i in range(20):
    paris_lat = paris_data2['geo_point_2d'][i][0:1]
    lis_lat.append(paris_lat)

In [51]:
lis_lon = []
for i in range(20):
    paris_lon = paris_data2['geo_point_2d'][i][1:2]
    lis_lon.append(paris_lon)

In [65]:
import itertools

In [77]:
df_lat = list(itertools.chain.from_iterable(lis_lat))
df_lon = list(itertools.chain.from_iterable(lis_lon))

In [78]:
df_lat

[48.87689616237872,
 48.86790337886785,
 48.85941549762748,
 48.86305413181178,
 48.84896809191946,
 48.854228281954754,
 48.8626304851685,
 48.88733716648682,
 48.87252726662346,
 48.82871768452136,
 48.83515623066034,
 48.844508659617546,
 48.88686862295828,
 48.86318677744551,
 48.87602855694339,
 48.86039876035177,
 48.892735074561706,
 48.85608259819694,
 48.84015541860987,
 48.82899321160942]

In [79]:
df_lon

[2.337460241388529,
 2.344107166658533,
 2.378741060237548,
 2.359361058970589,
 2.332670898588416,
 2.357361938142205,
 2.336293446550539,
 2.307485559493426,
 2.312582560420059,
 2.362468228516128,
 2.419807034965275,
 2.349859385560182,
 2.384694327870042,
 2.400819826729021,
 2.361112904561707,
 2.262099559395783,
 2.348711933867703,
 2.312438687733857,
 2.293559372435076,
 2.327100883257538]

In [73]:
#create a new list
paris_data3['Post_code'] = paris_data2['postal_code']
paris_data3['Name'] = paris_data2['nom_comm']
paris_data3['Town'] = paris_data2['nom_dept']

In [80]:
paris_data3['Latitude'] = df_lat
paris_data3['Longitude'] = df_lon
paris_data3

,Post_code,Name,Town,Latitude,Longitude
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671
5,75004,PARIS-4E-ARRONDISSEMENT,PARIS,48.854228,2.357362
6,75001,PARIS-1ER-ARRONDISSEMENT,PARIS,48.862630,2.336293
7,75017,PARIS-17E-ARRONDISSEMENT,PARIS,48.887337,2.307486
8,75008,PARIS-8E-ARRONDISSEMENT,PARIS,48.872527,2.312583
9,75013,PARIS-13E-ARRONDISSEMENT,PARIS,48.828718,2.362468


### 2. Visualization

#### 2.1 London

In [26]:
address = 'London, UK'
geolocator = Nominatim(user_agent="London_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('London are {}, {}.'.format(latitude, longitude))

London are 51.5073219, -0.1276474.


In [34]:
#Map of London
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

In [39]:
#add markers
for lat, lng, Town, Post_code in zip(london_data3['Latitude'], london_data3['Longitude'], london_data3['Town'], london_data3['Post_code']):
    label = '{}, {}'.format(Post_code, Town)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

#### 2.2 Paris

In [81]:
address = 'Paris, France'
geolocator = Nominatim(user_agent="Paris_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Paris are {}, {}.'.format(latitude, longitude))

Paris are 48.8566969, 2.3514616.


In [82]:
#Map of Paris
map_paris = folium.Map(location=[latitude, longitude], zoom_start=12)

In [84]:
#add markers
for lat, lng, Town, Name in zip(paris_data3['Latitude'], paris_data3['Longitude'], paris_data3['Town'], paris_data3['Name']):
    label = '{}, {}'.format(latitude, longitude)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_paris)  
    
map_paris

### 3.Venues

#### 3.1 London

In [130]:
Postcode_latitude = london_data3.loc[0, 'Latitude'] # neighborhood latitude value
Postcode_longitude = london_data3.loc[0, 'Longitude'] # neighborhood longitude value

Postcode_name = london_data3.loc[0, 'Post_code'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(Postcode_name, 
                                                               Postcode_latitude, 
                                                               Postcode_longitude))

Latitude and longitude values of E2 are 51.53179, -0.04385.


In [131]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Postcode_latitude, 
    Postcode_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LNK1UWBA5Y01WH44DTFMCGT4DZ4NLWOEP5YOFVYXGLGROD5Z&client_secret=Y0XT5AJR0SBKSBFEPZWR252RFSEV1INHGMOBOVSJ001ZDNOG&v=20180604&ll=51.53179,-0.04385&radius=500&limit=100'

In [132]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6022392ed28d0a02ec9a03cb'},
 'response': {'headerLocation': 'Bow West',
  'headerFullLocation': 'Bow West, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 26,
  'suggestedBounds': {'ne': {'lat': 51.5362900045,
    'lng': -0.036629706504375435},
   'sw': {'lat': 51.5272899955, 'lng': -0.051070293495624565}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '53713a7d11d263adba282dc5',
       'name': 'East London Liquor Company',
       'location': {'address': '221 Grove Road',
        'crossStreet': 'Bow Wharf',
        'lat': 51.53200460475683,
        'lng': -0.04150792650580123,
        'labeledLatLngs': [{'label': 'display',
          'lat': 51.53200460475683,
          'lng': -0.04150792650580123}],
        

In [133]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [134]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,East London Liquor Company,Cocktail Bar,51.532005,-0.041508
1,Pavilion Cafe,Café,51.533421,-0.042884
2,Palmers Restaurant,Restaurant,51.530222,-0.042782
3,The Crown,Pub,51.533182,-0.041577
4,Simplyfresh,Convenience Store,51.529435,-0.046022


In [135]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

26 venues were returned by Foursquare.


In [136]:
#all London area
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [137]:
london_venues = getNearbyVenues(names=london_data3['Post_code'],
                                   latitudes=london_data3['Latitude'],
                                   longitudes=london_data3['Longitude']
                                  )

E2
E3
E4
E5
E6
E7
E8
E9
E10
E11
E12
E13
E14
E15
E16
E17
E18
E20
E1
E1W
EC1A
EC1M
EC1N
EC1R
EC1V
EC1Y
EC1A
EC1M
EC1N
EC1R
EC1V
EC1Y
EC2A
EC2M
EC2N
EC2R
EC2V
EC2Y
EC3A
EC3M
EC3N
EC3R
EC3V
EC4A
EC4M
EC4N
EC4R
EC4V
EC4Y
N2
N3
N4
N5
N6
N7
N8
N9
N10
N11
N12
N13
N14
N15
N16
N17
N18
N19
N20
N21
N22
N1
N1C
NW1
NW2
NW3
NW4
NW5
NW6
NW7
NW8
NW9
NW10
NW11
SE1
SE2
SE3
SE4
SE5
SE6
SE7
SE8
SE9
SE10
SE11
SE12
SE13
SE14
SE15
SE16
SE17
SE18
SE19
SE20
SE21
SE22
SE23
SE24
SE25
SE26
SE27
SE28
SW2
SW3
SW4
SW5
SW6
SW7
SW8
SW9
SW10
SW11
SW12
SW13
SW14
SW15
SW16
SW17
SW18
SW19
SW20
SW1A
SW1E
SW1H
SW1P
SW1V
SW1W
SW1X
SW1Y
W2
W3
W4
W5
W6
W7
W8
W9
W10
W11
W12
W13
W14
W1B
W1C
W1D
W1F
W1G
W1H
W1J
W1K
W1S
W1T
W1U
W1W
WC1A
WC1B
WC1E
WC1H
WC1N
WC1R
WC1V
WC1X
WC2A
WC2B
WC2E
WC2H
WC2N
WC2R


In [138]:
print(london_venues.shape)
london_venues.head()

(6942, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,E2,51.53179,-0.04385,East London Liquor Company,51.532005,-0.041508,Cocktail Bar
1,E2,51.53179,-0.04385,Pavilion Cafe,51.533421,-0.042884,Café
2,E2,51.53179,-0.04385,Palmers Restaurant,51.530222,-0.042782,Restaurant
3,E2,51.53179,-0.04385,The Crown,51.533182,-0.041577,Pub
4,E2,51.53179,-0.04385,Simplyfresh,51.529435,-0.046022,Convenience Store


In [139]:
london_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
E1,13,13,13,13,13,13
E10,26,26,26,26,26,26
E11,4,4,4,4,4,4
E12,5,5,5,5,5,5
E13,8,8,8,8,8,8
...,...,...,...,...,...,...
WC2B,100,100,100,100,100,100
WC2E,64,64,64,64,64,64
WC2H,100,100,100,100,100,100


In [140]:
print('There are {} uniques categories.'.format(len(london_venues['Venue Category'].unique())))

There are 336 uniques categories.


#### 3.2 Paris

In [141]:
Postcode_latitude = paris_data3.loc[0, 'Latitude'] # neighborhood latitude value
Postcode_longitude = paris_data3.loc[0, 'Longitude'] # neighborhood longitude value

Postcode_name = paris_data3.loc[0, 'Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(Postcode_name, 
                                                               Postcode_latitude, 
                                                               Postcode_longitude))

Latitude and longitude values of PARIS-9E-ARRONDISSEMENT are 48.87689616237872, 2.337460241388529.


In [142]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Postcode_latitude, 
    Postcode_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LNK1UWBA5Y01WH44DTFMCGT4DZ4NLWOEP5YOFVYXGLGROD5Z&client_secret=Y0XT5AJR0SBKSBFEPZWR252RFSEV1INHGMOBOVSJ001ZDNOG&v=20180604&ll=48.87689616237872,2.337460241388529&radius=500&limit=100'

In [143]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6022398203e3b90664ca5dd1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Saint Georges',
  'headerFullLocation': 'Saint Georges, Paris',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 148,
  'suggestedBounds': {'ne': {'lat': 48.88139616687872,
    'lng': 2.3442897160747966},
   'sw': {'lat': 48.87239615787872, 'lng': 2.330630766702261}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5810d00e38fa41678dcb1ac5',
       'name': 'So Nat',
       'location': {'address': '5 rue Bourdaloue',
        'lat': 48.87627724467217,
        'lng': 2.3386138527194253,
        'labeledLatLngs': [{'label': 'display',
          'lat': 48.8762

In [144]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [145]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,So Nat,Vegetarian / Vegan Restaurant,48.876277,2.338614
1,RAP,Gourmet Shop,48.876628,2.339359
2,Le Bouclier de Bacchus,Wine Bar,48.876834,2.337843
3,Farine & O,Bakery,48.877209,2.339464
4,Place Saint-Georges,Plaza,48.878408,2.337575


In [146]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [147]:
#all Paris area
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [148]:
paris_venues = getNearbyVenues(names=paris_data3['Name'],
                                   latitudes=paris_data3['Latitude'],
                                   longitudes=paris_data3['Longitude']
                                  )

PARIS-9E-ARRONDISSEMENT
PARIS-2E-ARRONDISSEMENT
PARIS-11E-ARRONDISSEMENT
PARIS-3E-ARRONDISSEMENT
PARIS-6E-ARRONDISSEMENT
PARIS-4E-ARRONDISSEMENT
PARIS-1ER-ARRONDISSEMENT
PARIS-17E-ARRONDISSEMENT
PARIS-8E-ARRONDISSEMENT
PARIS-13E-ARRONDISSEMENT
PARIS-12E-ARRONDISSEMENT
PARIS-5E-ARRONDISSEMENT
PARIS-19E-ARRONDISSEMENT
PARIS-20E-ARRONDISSEMENT
PARIS-10E-ARRONDISSEMENT
PARIS-16E-ARRONDISSEMENT
PARIS-18E-ARRONDISSEMENT
PARIS-7E-ARRONDISSEMENT
PARIS-15E-ARRONDISSEMENT
PARIS-14E-ARRONDISSEMENT


In [149]:
print(paris_venues.shape)
paris_venues.head()

(1279, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,So Nat,48.876277,2.338614,Vegetarian / Vegan Restaurant
1,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,RAP,48.876628,2.339359,Gourmet Shop
2,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Le Bouclier de Bacchus,48.876834,2.337843,Wine Bar
3,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Farine & O,48.877209,2.339464,Bakery
4,PARIS-9E-ARRONDISSEMENT,48.876896,2.33746,Place Saint-Georges,48.878408,2.337575,Plaza


In [150]:
paris_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
PARIS-10E-ARRONDISSEMENT,100,100,100,100,100,100
PARIS-11E-ARRONDISSEMENT,39,39,39,39,39,39
PARIS-12E-ARRONDISSEMENT,5,5,5,5,5,5
PARIS-13E-ARRONDISSEMENT,63,63,63,63,63,63
PARIS-14E-ARRONDISSEMENT,27,27,27,27,27,27
PARIS-15E-ARRONDISSEMENT,62,62,62,62,62,62
PARIS-16E-ARRONDISSEMENT,11,11,11,11,11,11
PARIS-17E-ARRONDISSEMENT,65,65,65,65,65,65
PARIS-18E-ARRONDISSEMENT,70,70,70,70,70,70


In [151]:
print('There are {} uniques categories.'.format(len(paris_venues['Venue Category'].unique())))

There are 204 uniques categories.


### 4. One Hot Encoding

#### 4.1 London

In [152]:
# one hot encoding
london_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
london_onehot['Neighborhood'] = london_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [london_onehot.columns[-1]] + list(london_onehot.columns[:-1])
london_onehot = london_onehot[fixed_columns]

london_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
london_onehot.shape

(6942, 336)

In [154]:
london_grouped = london_onehot.groupby('Neighborhood').mean().reset_index()
london_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,E1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
1,E10,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
2,E11,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
3,E12,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
4,E13,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,WC2B,0.0,0.0,0.0,0.010000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.01,0.0,0.01,0.0
167,WC2E,0.0,0.0,0.0,0.015625,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.015625,0.00,0.0,0.00,0.0
168,WC2H,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.00,0.0
169,WC2N,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.00,0.0,0.00,0.0


In [155]:
london_grouped.shape

(171, 336)

In [195]:
num_top_venues = 5

for hood in london_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----E1----
           venue  freq
0  Grocery Store  0.23
1            Pub  0.15
2           Park  0.15
3    Coffee Shop  0.08
4           Pool  0.08


----E10----
                  venue  freq
0                   Pub  0.12
1              Platform  0.08
2           Coffee Shop  0.08
3   Fried Chicken Joint  0.08
4  Fast Food Restaurant  0.08


----E11----
                           venue  freq
0             Chinese Restaurant  0.50
1              Fish & Chips Shop  0.25
2                            Pub  0.25
3                    Yoga Studio  0.00
4  Paper / Office Supplies Store  0.00


----E12----
                  venue  freq
0     Indian Restaurant   0.4
1         Train Station   0.2
2    Turkish Restaurant   0.2
3  Gym / Fitness Center   0.2
4           Yoga Studio   0.0


----E13----
               venue  freq
0           Bus Stop  0.12
1     Breakfast Spot  0.12
2           Boutique  0.12
3  Convenience Store  0.12
4        Rugby Pitch  0.12


----E14----
                venue  fr

In [196]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [227]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = london_grouped['Neighborhood']

for ind in np.arange(london_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,E1,Grocery Store,Pub,Park,Pool,Coffee Shop,Snack Place,Street Art,Bus Stop,Italian Restaurant,Design Studio
1,E10,Pub,Fried Chicken Joint,Coffee Shop,Platform,Fast Food Restaurant,Bus Stop,Shopping Plaza,Metro Station,Café,Optical Shop
2,E11,Chinese Restaurant,Pub,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
3,E12,Indian Restaurant,Turkish Restaurant,Gym / Fitness Center,Train Station,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
4,E13,Fried Chicken Joint,Convenience Store,Bus Station,Breakfast Spot,Boutique,Rugby Pitch,Asian Restaurant,Bus Stop,Distillery,Flower Shop


In [228]:
# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 4, 2, 2, 2, 2, 2, 1, 0], dtype=int32)

In [229]:
london_merged

,Post_code,Town,Latitude,Longitude
0,E2,London,51.53179,-0.04385
1,E3,London,51.53952,-0.03401
2,E4,London,51.63223,0.02200
3,E5,London,51.55989,-0.03337
4,E6,London,51.53823,0.04204
...,...,...,...,...
172,WC2B,London,51.51490,-0.11847
173,WC2E,London,51.51091,-0.12033
174,WC2H,London,51.51265,-0.12908
175,WC2N,London,51.51025,-0.12780


In [230]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

london_merged = london_data3

london_merged = london_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Post_code')

london_merged.head() # check the last columns!

,Post_code,Town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,E2,London,51.53179,-0.04385,4,Pub,Café,Dance Studio,Convenience Store,Art Gallery,Park,Cocktail Bar,Middle Eastern Restaurant,Canal,Italian Restaurant
1,E3,London,51.53952,-0.03401,4,Pub,Garden,Skate Park,Tennis Court,Plaza,Antique Shop,Park,Coffee Shop,Garden Center,Event Space
2,E4,London,51.63223,0.02200,4,Pub,History Museum,Café,Used Auto Dealership,Hotel,Scenic Lookout,Flea Market,Flower Shop,Escape Room,Ethiopian Restaurant
3,E5,London,51.55989,-0.03337,2,Cricket Ground,Nature Preserve,Travel & Transport,Athletics & Sports,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
4,E6,London,51.53823,0.04204,3,Grocery Store,Indian Restaurant,Tea Room,Park,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [231]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_merged['Latitude'], london_merged['Longitude'], london_merged['Post_code'], london_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [232]:
london_merged.loc[london_merged['Cluster Labels'] == 0, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,London,Park,Playground,Bistro,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
16,London,Park,Warehouse Store,Trail,Home Service,Food & Drink Shop,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
56,London,Park,Pub,Bakery,Fast Food Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
65,London,Park,Pizza Place,Fish & Chips Shop,Soccer Field,Women's Store,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
68,London,Park,Playground,Golf Course,Outdoor Gym,Women's Store,Farmers Market,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
74,London,Scenic Lookout,Park,Track,Lake,Tree,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit
80,London,Bar,Construction & Landscaping,Auto Workshop,Park,Food Court,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
104,London,Park,Beer Store,Pub,Ice Cream Shop,Coffee Shop,Convenience Store,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit
107,London,Park,Filipino Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
127,London,Gym,Music Venue,Hotel,Park,Women's Store,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [233]:
london_merged.loc[london_merged['Cluster Labels'] == 1, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,London,Forest,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
84,London,Forest,Construction & Landscaping,Film Studio,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [234]:
london_merged.loc[london_merged['Cluster Labels'] == 2, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,London,Cricket Ground,Nature Preserve,Travel & Transport,Athletics & Sports,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
6,London,Café,Turkish Restaurant,Grocery Store,Coffee Shop,Pub,Bar,Convenience Store,Cocktail Bar,Restaurant,Tapas Restaurant
10,London,Indian Restaurant,Turkish Restaurant,Gym / Fitness Center,Train Station,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
11,London,Fried Chicken Joint,Convenience Store,Bus Station,Breakfast Spot,Boutique,Rugby Pitch,Asian Restaurant,Bus Stop,Distillery,Flower Shop
12,London,Chinese Restaurant,Outdoor Sculpture,Canal Lock,Women's Store,Filipino Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
...,...,...,...,...,...,...,...,...,...,...,...
172,London,Coffee Shop,Pub,Theater,Hotel,Restaurant,Burger Joint,Bakery,Dessert Shop,Boutique,Café
173,London,Theater,Burger Joint,Steakhouse,Restaurant,Pub,Coffee Shop,Cocktail Bar,Historic Site,Opera House,Café
174,London,Theater,Bakery,Ice Cream Shop,Steakhouse,Coffee Shop,Seafood Restaurant,Plaza,Burger Joint,Liquor Store,Sandwich Place
175,London,Theater,Hotel,Bakery,Plaza,Ice Cream Shop,Steakhouse,Coffee Shop,Japanese Restaurant,Cocktail Bar,Boutique


In [235]:
london_merged.loc[london_merged['Cluster Labels'] == 3, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,London,Grocery Store,Indian Restaurant,Tea Room,Park,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
18,London,Grocery Store,Pub,Park,Pool,Coffee Shop,Snack Place,Street Art,Bus Stop,Italian Restaurant,Design Studio
53,London,Café,History Museum,Park,Tea Room,Lake,Filipino Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm
58,London,Bus Stop,Park,Café,Rental Car Location,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
59,London,Grocery Store,Italian Restaurant,Dessert Shop,Women's Store,Filipino Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
61,London,Café,Greek Restaurant,Grocery Store,Bus Stop,Paper / Office Supplies Store,Metro Station,Fast Food Restaurant,Golf Course,Pub,Indian Restaurant
78,London,Bus Stop,Farm,Café,Women's Store,Film Studio,Exhibit,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant
82,London,Grocery Store,Sandwich Place,Tennis Court,Park,Trail,Middle Eastern Restaurant,Farm,Ethiopian Restaurant,Event Space,Exhibit
88,London,Park,Grocery Store,Gym / Fitness Center,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
110,London,Grocery Store,Fast Food Restaurant,Pub,Bus Station,Gym / Fitness Center,Bakery,Event Space,Exhibit,Fabric Shop,Falafel Restaurant


In [236]:
london_merged.loc[london_merged['Cluster Labels'] == 4, london_merged.columns[[1] + list(range(5, london_merged.shape[1]))]]

,Town,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,London,Pub,Café,Dance Studio,Convenience Store,Art Gallery,Park,Cocktail Bar,Middle Eastern Restaurant,Canal,Italian Restaurant
1,London,Pub,Garden,Skate Park,Tennis Court,Plaza,Antique Shop,Park,Coffee Shop,Garden Center,Event Space
2,London,Pub,History Museum,Café,Used Auto Dealership,Hotel,Scenic Lookout,Flea Market,Flower Shop,Escape Room,Ethiopian Restaurant
5,London,Pub,Bar,Bakery,Restaurant,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
8,London,Pub,Fried Chicken Joint,Coffee Shop,Platform,Fast Food Restaurant,Bus Stop,Shopping Plaza,Metro Station,Café,Optical Shop
9,London,Chinese Restaurant,Pub,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
23,London,Pub,Coffee Shop,Café,Vietnamese Restaurant,Bar,Italian Restaurant,Pizza Place,Park,Hotel,French Restaurant
29,London,Pub,Coffee Shop,Café,Vietnamese Restaurant,Bar,Italian Restaurant,Pizza Place,Park,Hotel,French Restaurant
43,London,Pub,Coffee Shop,Sandwich Place,Italian Restaurant,Wine Bar,Burrito Place,French Restaurant,Sushi Restaurant,Japanese Restaurant,Park
52,London,Café,Pub,Coffee Shop,Soccer Stadium,French Restaurant,Bar,Gym / Fitness Center,Park,Yoga Studio,Ethiopian Restaurant


#### 4.2 Paris

In [237]:
# one hot encoding
paris_onehot = pd.get_dummies(london_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
paris_onehot['Neighborhood'] = paris_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [paris_onehot.columns[-1]] + list(paris_onehot.columns[:-1])
paris_onehot = paris_onehot[fixed_columns]

paris_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [239]:
paris_grouped = paris_onehot.groupby('Neighborhood').mean().reset_index()
paris_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Watch Shop,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store
0,PARIS-10E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.01,...,0.00,0.020000,0.000000,0.0,0.000000,0.020000,0.010000,0.0,0.0,0.0
1,PARIS-11E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.025641,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
2,PARIS-12E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
3,PARIS-13E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.015873,0.000000,0.0,0.000000,0.015873,0.000000,0.0,0.0,0.0
4,PARIS-14E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.037037,0.0,0.0,0.0
5,PARIS-15E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.000000,0.0,0.032258,0.064516,0.016129,0.0,0.0,0.0
6,PARIS-16E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
7,PARIS-17E-ARRONDISSEMENT,0.00,0.000000,0.015385,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.030769,0.000000,0.0,0.000000,0.000000,0.015385,0.0,0.0,0.0
8,PARIS-18E-ARRONDISSEMENT,0.00,0.000000,0.014286,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.085714,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
9,PARIS-19E-ARRONDISSEMENT,0.00,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.00,...,0.00,0.000000,0.000000,0.0,0.022222,0.000000,0.000000,0.0,0.0,0.0


In [240]:
num_top_venues = 5

for hood in paris_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = paris_grouped[paris_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----PARIS-10E-ARRONDISSEMENT----
                venue  freq
0                 Pub  0.12
1         Coffee Shop  0.07
2    Sushi Restaurant  0.06
3  Italian Restaurant  0.06
4      Sandwich Place  0.05


----PARIS-11E-ARRONDISSEMENT----
           venue  freq
0           Park  0.10
1    Coffee Shop  0.10
2  Grocery Store  0.08
3       Platform  0.05
4           Pool  0.03


----PARIS-12E-ARRONDISSEMENT----
                   venue  freq
0            Coffee Shop   0.4
1           Concert Hall   0.2
2            Art Gallery   0.2
3  Performing Arts Venue   0.2
4            Yoga Studio   0.0


----PARIS-13E-ARRONDISSEMENT----
         venue  freq
0   Food Truck  0.11
1  Coffee Shop  0.10
2          Pub  0.10
3         Park  0.05
4         Café  0.05


----PARIS-14E-ARRONDISSEMENT----
              venue  freq
0       Coffee Shop  0.07
1        Restaurant  0.07
2        Food Truck  0.07
3        Boxing Gym  0.04
4  Sushi Restaurant  0.04


----PARIS-15E-ARRONDISSEMENT----
                  

In [241]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [242]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = paris_grouped['Neighborhood']

for ind in np.arange(paris_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-10E-ARRONDISSEMENT,Grocery Store,Pub,Park,Pool,Coffee Shop,Snack Place,Street Art,Bus Stop,Italian Restaurant,Design Studio
1,PARIS-11E-ARRONDISSEMENT,Pub,Fried Chicken Joint,Coffee Shop,Platform,Fast Food Restaurant,Bus Stop,Shopping Plaza,Metro Station,Café,Optical Shop
2,PARIS-12E-ARRONDISSEMENT,Chinese Restaurant,Pub,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
3,PARIS-13E-ARRONDISSEMENT,Indian Restaurant,Turkish Restaurant,Gym / Fitness Center,Train Station,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
4,PARIS-14E-ARRONDISSEMENT,Fried Chicken Joint,Convenience Store,Bus Station,Breakfast Spot,Boutique,Rugby Pitch,Asian Restaurant,Bus Stop,Distillery,Flower Shop


In [243]:
# set number of clusters
kclusters = 5

paris_grouped_clustering = paris_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(paris_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 1, 4, 2, 2, 3, 4, 4, 2], dtype=int32)

In [244]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

paris_merged = paris_data3

paris_merged = paris_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Name')

paris_merged.head() # check the last columns!

,Post_code,Name,Town,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,75009,PARIS-9E-ARRONDISSEMENT,PARIS,48.876896,2.337460,4,Park,Playground,Bistro,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
1,75002,PARIS-2E-ARRONDISSEMENT,PARIS,48.867903,2.344107,0,Clothing Store,Burger Joint,Juice Bar,Cosmetics Shop,Department Store,Toy / Game Store,Italian Restaurant,Bar,Food Court,Mexican Restaurant
2,75011,PARIS-11E-ARRONDISSEMENT,PARIS,48.859415,2.378741,0,Pub,Fried Chicken Joint,Coffee Shop,Platform,Fast Food Restaurant,Bus Stop,Shopping Plaza,Metro Station,Café,Optical Shop
3,75003,PARIS-3E-ARRONDISSEMENT,PARIS,48.863054,2.359361,0,Pub,Garden,Skate Park,Tennis Court,Plaza,Antique Shop,Park,Coffee Shop,Garden Center,Event Space
4,75006,PARIS-6E-ARRONDISSEMENT,PARIS,48.848968,2.332671,2,Grocery Store,Indian Restaurant,Tea Room,Park,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [245]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(paris_merged['Latitude'], paris_merged['Longitude'], paris_merged['Post_code'], paris_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [256]:
paris_merged.loc[paris_merged['Cluster Labels'] == 0, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,PARIS-2E-ARRONDISSEMENT,0,Clothing Store,Burger Joint,Juice Bar,Cosmetics Shop,Department Store,Toy / Game Store,Italian Restaurant,Bar,Food Court,Mexican Restaurant
2,PARIS-11E-ARRONDISSEMENT,0,Pub,Fried Chicken Joint,Coffee Shop,Platform,Fast Food Restaurant,Bus Stop,Shopping Plaza,Metro Station,Café,Optical Shop
3,PARIS-3E-ARRONDISSEMENT,0,Pub,Garden,Skate Park,Tennis Court,Plaza,Antique Shop,Park,Coffee Shop,Garden Center,Event Space


In [255]:
paris_merged.loc[paris_merged['Cluster Labels'] == 1, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,PARIS-12E-ARRONDISSEMENT,1,Chinese Restaurant,Pub,Fish & Chips Shop,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm


In [254]:
paris_merged.loc[paris_merged['Cluster Labels'] == 2, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,PARIS-6E-ARRONDISSEMENT,2,Grocery Store,Indian Restaurant,Tea Room,Park,Farmers Market,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
11,PARIS-5E-ARRONDISSEMENT,2,Cricket Ground,Nature Preserve,Travel & Transport,Athletics & Sports,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant
12,PARIS-19E-ARRONDISSEMENT,2,Park,Warehouse Store,Trail,Home Service,Food & Drink Shop,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
17,PARIS-7E-ARRONDISSEMENT,2,Pub,Bar,Bakery,Restaurant,Women's Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
18,PARIS-15E-ARRONDISSEMENT,2,Chinese Restaurant,Outdoor Sculpture,Canal Lock,Women's Store,Filipino Restaurant,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market
19,PARIS-14E-ARRONDISSEMENT,2,Fried Chicken Joint,Convenience Store,Bus Station,Breakfast Spot,Boutique,Rugby Pitch,Asian Restaurant,Bus Stop,Distillery,Flower Shop


In [253]:
paris_merged.loc[paris_merged['Cluster Labels'] == 3, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,PARIS-16E-ARRONDISSEMENT,3,Grocery Store,Indian Restaurant,Coffee Shop,Restaurant,Beer Garden,Farmers Market,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop


In [252]:
paris_merged.loc[paris_merged['Cluster Labels'] == 4, paris_merged.columns[[1] + list(range(5, paris_merged.shape[1]))]]

,Name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS-9E-ARRONDISSEMENT,4,Park,Playground,Bistro,Women's Store,Fast Food Restaurant,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm
5,PARIS-4E-ARRONDISSEMENT,4,Pub,History Museum,Café,Used Auto Dealership,Hotel,Scenic Lookout,Flea Market,Flower Shop,Escape Room,Ethiopian Restaurant
6,PARIS-1ER-ARRONDISSEMENT,4,Hotel,Gym / Fitness Center,Sandwich Place,Coffee Shop,Castle,Pub,History Museum,Indian Restaurant,Garden,General Entertainment
7,PARIS-17E-ARRONDISSEMENT,4,Coffee Shop,Park,Platform,Nature Preserve,Light Rail Station,Supermarket,Grocery Store,Skate Park,Turkish Restaurant,Hardware Store
8,PARIS-8E-ARRONDISSEMENT,4,Café,Turkish Restaurant,Grocery Store,Coffee Shop,Pub,Bar,Convenience Store,Cocktail Bar,Restaurant,Tapas Restaurant
9,PARIS-13E-ARRONDISSEMENT,4,Indian Restaurant,Turkish Restaurant,Gym / Fitness Center,Train Station,Women's Store,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Fabric Shop
13,PARIS-20E-ARRONDISSEMENT,4,Pub,Café,Dance Studio,Convenience Store,Art Gallery,Park,Cocktail Bar,Middle Eastern Restaurant,Canal,Italian Restaurant
14,PARIS-10E-ARRONDISSEMENT,4,Grocery Store,Pub,Park,Pool,Coffee Shop,Snack Place,Street Art,Bus Stop,Italian Restaurant,Design Studio
16,PARIS-18E-ARRONDISSEMENT,4,Forest,Escape Room,Event Space,Exhibit,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


### 5. Result and Conclustion

#### London and Paris show very multicultural character. From the venues list, we can see both cities have many different kinds of cuisines. also we can see the there are not only restaurants but also hotels, cafe, touristic places, grocery stores, or so, which we can tell you can get and see in the city. 
#### as a conclusion of this project, both cities show pretty much similar variety of business. and both city offer not only business opportunity but also touristic places.